<a href="https://colab.research.google.com/github/PuppyQ08/CUDA-in-notebook/blob/main/vecadd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi I am QQY. In this session, we are going to write CUDA codes related to the vector adding. It is pretty straightforward and we can get to learn how CUDA manipulate data on GPU and how CUDA Nsight compute provide you helpful insight.
We would use [nvcc4jupyter](https://github.com/andreinechaev/nvcc4jupyter) tool to run CUDA code in colab environment.

In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpspo8rwcz".
--2025-12-26 18:03:44--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2025-12-26 18:03:44--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

c

This is the util file and you can add any helper function you need here.

In [3]:
%%cpp -n until.h
#include <math.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

In [8]:
%%cuda_group_save --group shared --name "utils.h"
#include <math.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

This is the main body of files.

In [ ]:
%%cuda_group_save --name "vector_add.cu" --group "vector_add"
// vector add kernel: C = A + B
template <typename T>
__global__ void vadd(const float *A, const float *B, float *C, int ds){
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < ds) {
        C[idx] = A[idx] + B[idx];
    }
}